# 最適化
optimizerを用いたパラメータの最適化(学習)について。

# パラメータの最適化
モデルに対して、訓練データを用いてパラメータを最適化する。  
モデルの訓練は反復的なプロセスとなる。  
1. モデルは出力を計算し、損失を求める。  
2. 各パラメータに対して、損失に対する偏微分の値を計算する。  
3. 勾配降下法に基づいてパラメータを最適化する。  
4. 各イテレーション(エポック)で繰り返す。  

最適化のプロセスは次の動画も参照[最適化プロセス](https://www.youtube.com/watch?v=tIeHLnjs5U8)

In [ ]:
# コードの準備
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data,  batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork()


# ハイパーパラメータ
ハイパーパラメータとは、モデルの最適化プロセスを制御するためのパラメータである。  
ハイパーパラメータの値は、モデルの学習や収束率に影響する。  
詳細は[ハイパーパラメータの詳細](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)を参照  


今回は、訓練用のハイパーパラメータとして、
1. **Number of Epochs** : epoch数  
2. **Batch Size** : ミニバッチサイズを構成するデータサイズ  
3. **Leaning Rate** : パラメータの更新の係数

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# 最適化ループ
ハイパーパラメータの設定後、訓練で、最適化のループを回すことで、モデルを最適化する。  
最適化ループの1回のイテレーションのことを**エポック(epoch)**という。  
各エポックは以下の2種類のループから構成される。  
1. **訓練ループ** : データセットに対して訓練を実行しパラメータを収束させる。  
2. **検証/テストループ** : テストデータでモデルを評価し、性能を評価する  

# 損失関数 : Loss Function
データが与えられただけでは、その予測結果と実際の結果があっているのかどうかが不明。  
そこで、損失関数を導入することで、  
推論した結果と実際の結果の誤差の大きさを測定することで、予測精度を上げていく。  

一般的な損失関数としては、  
回帰タスクでは、`nn.MSELoss(Mean Square Error)`  
分類タスクでは、`nn.NLLLOSS(Negative Log Likelihood)`が使用される。  
`nn.CrossEntropyLoss`は、`nn.LogSoftmax` と`nn.NLLLoss`を結合した損失関数となる。  
モデルが出力する`logits`を`nn.CrossEntropyLoss`に与えて正規化し、予測誤差を求める。

In [ ]:
# 損失関数の初期化
loss_fn = nn.CrossEntropyLoss()

# 最適化 : Optimizer
最適化は各訓練ステップにおいてモデルの誤差を小さくするようにパラメータを調整するステップ  

# 最適化アルゴリズム : Optimization algorithms
最適化アルゴリズムとは、最適化プロセスの具体的な手続きのこと。  
チュートリアルでは、確率的勾配降下法(`SGD : Stochastic Gradient Descent`)を用いる。  
最適化のロジックは全て`optimizer`オブジェクト内に含まれている。  
最適化の詳細については[最適化の詳細](https://pytorch.org/docs/stable/optim.html)を参照  

# optimizerの初期化
1. 訓練したいモデルパラメータをoptimizerに登録(渡す)する  
2. 同時に、学習率をハイパーパラメータとして渡す。  

# optimizerの実装
初期化後、最適化(optimizer)は3つのステップから構成される。  
1. `optimizer.zero_grad()`を実行し、モデルパラメータの勾配をリセットする。  
    > 勾配の計算は蓄積されていくので、epochごとに明示的にリセットする
2. 次に、`loss.backward()`を実行し、バックプロパゲーションを実行する。  
    > 損失に対する各ぱらめーたの勾配を求める。
3. 最後に、`optimizer.step()`を実行し、パラメータを更新する。
    > 各パラメータの勾配を用いて、値を更新する。

# 実装について
最適化を実行するコードをループする関数として、`train`を、  
テストデータに対する予測をする関数として、`test`を定義する。

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        # 予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss : {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    # testデータでは勾配は不要
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    
    test_loss /= size
    correct /= size
    print(f"Test Error: Accuracy {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

# 実行
損失関数とoptimizerを初期化し、train,testそれぞれに渡す。

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epoch = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

# 詳細
損失関数や最適化などの詳細は、  
[Loss Function](https://pytorch.org/docs/stable/nn.html#loss-functions)  
[torch.optim](https://pytorch.org/docs/stable/optim.html)  
[Warmstart Trainig a Model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)  
を参照